In [119]:
# This notebooks is an attempt to port the code form https://github.com/MostHumble/Clinical-GAN/blob/master/process_data.py 
# to a more recent mimic dataset version
# while adding suitable updates that weren't taken into account: for now mailnly scheduled to work on stratification

In [53]:
import sys
import pickle
from datetime import datetime
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import torch
from collections import Counter
import argparse
import gzip

#parser = argparse.ArgumentParser()

In [2]:
!ls mimic-iv-2.2/hosp

admissions.csv.gz	 emar_detail.csv.gz	    poe_detail.csv.gz
d_hcpcs.csv.gz		 hcpcsevents.csv.gz	    prescriptions.csv.gz
diagnoses_icd.csv.gz	 labevents.csv.gz	    procedures_icd.csv.gz
d_icd_diagnoses.csv.gz	 microbiologyevents.csv.gz  provider.csv.gz
d_icd_procedures.csv.gz  omr.csv.gz		    services.csv.gz
d_labitems.csv.gz	 patients.csv.gz	    transfers.csv.gz
drgcodes.csv.gz		 pharmacy.csv.gz
emar.csv.gz		 poe.csv.gz


In [3]:
mimic_iv_path = 'mimic-iv-2.2/hosp'
CCSRDX_file = 'DXCCSR_v2021-2/DXCCSR_v2021-2.csv'
CCSRPCS_file = 'PRCCSR_v2021-1/PRCCSR_v2021-1.csv'
#os.path.join(mimic_iv_path, 'ADMISSIONS.csv')
admissionFile = os.path.join(mimic_iv_path, 'admissions.csv.gz')
diagnosisFile = os.path.join(mimic_iv_path, 'diagnoses_icd.csv.gz')
procedureFile = os.path.join(mimic_iv_path, 'procedures_icd.csv.gz')
#patientsAge = os.path.join(mimic_iv_path, 'patientsAge.csv')
prescriptionFile = os.path.join(mimic_iv_path, 'prescriptions.csv.gz')
#diagnosisFrequencyFile = os.path.join(mimic_iv_path, 'WITHOUT_IF_CODE_COUNT.csv')
#outFile = 'data'

In [4]:
adm = pd.read_csv(admissionFile)

In [5]:
adm.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [6]:
def reformat_icd(code: str, version: int, is_diag: bool) -> str:
    """format icd code depending on version"""
    if version == 9:
        return reformat_icd9(code, is_diag)
    elif version == 10:
        return reformat_icd10(code, is_diag)
    else:
        raise ValueError("version must be 9 or 10")

def reformat_icd10(code: str, is_diag: bool) -> str:
    """
    Put a period in the right place because the MIMIC-3 data files exclude them.
    Generally, procedure codes have dots after the first two digits,
    while diagnosis codes have dots after the first three digits.
    """
    code = "".join(code.split("."))
    if not is_diag:
        return code
    return code[:3] + "." + code[3:]


def reformat_icd9(code: str, is_diag: bool) -> str:
    """
    Put a period in the right place because the MIMIC-3 data files exclude them.
    Generally, procedure codes have dots after the first two digits,
    while diagnosis codes have dots after the first three digits.
    """
    code = "".join(code.split("."))
    if is_diag:
        if code.startswith("E"):
            if len(code) > 4:
                return code[:4] + "." + code[4:]
        else:
            if len(code) > 3:
                return code[:3] + "." + code[3:]
    else:
        if len(code) > 2:
            return code[:2] + "." + code[2:]
    return code


In [45]:
diag = pd.read_csv(diagnosisFile)

In [46]:
diag.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


In [24]:
diag.columns.get_loc('icd_code')

3

In [25]:
proc = pd.read_csv(procedureFile)
proc.head()

,subject_id,hadm_id,seq_num,chartdate,icd_code,icd_version
0,10000032,22595853,1,2180-05-07,5491,9
1,10000032,22841357,1,2180-06-27,5491,9
2,10000032,25742920,1,2180-08-06,5491,9
3,10000068,25022803,1,2160-03-03,8938,9
4,10000117,27988844,1,2183-09-19,0QS734Z,10


In [26]:
proc.columns.get_loc('icd_code')

4

In [110]:
def get_ICDs_from_mimic_file(fileName ,isdiagnosis=True):
    
    mapping = {}
    mimicFile = gzip.open(fileName, 'r')
        
    codes = []
    
    number_of_null_ICD9_codes = 0
    number_of_null_ICD10_codes = 0
    mimicFile.readline()
    for line in mimicFile:  #   0  ,     1    ,    2   ,   3  ,    4
        tokens = line.decode('utf-8').strip().split(',')
        #print(tokens)
        hadm_id = int(tokens[1])
        if ( isdiagnosis and len(tokens[3]) == 0) or ( not(isdiagnosis) and len(tokens[4]) == 0 ):
            if isdiagnosis:
                if (tokens[4] =='9'):
                    # ignore diagnoses where ICD9_code is null
                    number_of_null_ICD9_codes += 1
                else:
                    number_of_null_ICD10_codes += 1

                continue;
            else:
                if (tokens[5] =='9'):
                    # ignore diagnoses where ICD9_code is null
                    number_of_null_ICD9_codes += 1
                else:
                    number_of_null_ICD10_codes += 1

                continue;
                
        if isdiagnosis:
            ICD_code = tokens[3]
        else:
            ICD_code = tokens[4] 
            
            
        if ICD_code.find("\"") != -1:
            #print("ICD_Code before",ICD_code )
            ICD_code = ICD_code[1:-1].strip()  # toss off quotes and proceed
            #print("ICD_Code after",ICD_code )
        # since diagnosis and procedure ICD9 codes have intersections, a prefix is necessary for disambiguation
       
        if isdiagnosis:
            ICD_code = 'D' + tokens[4]+ '_' +ICD_code
        else:
            ICD_code = 'P' + tokens[5] + '_' + ICD_code

        # To understand the line below, check https://mimic.physionet.org/mimictables/diagnoses_icd/
        # "The code field for the ICD-9-CM Principal and Other Diagnosis Codes is six characters in length (not really!),
        # with the decimal point implied between the third and fourth digit for all diagnosis codes other than the V codes.
        # The decimal is implied for V codes between the second and third digit."
        # Actually, if you look at the codes (https://raw.githubusercontent.com/drobbins/ICD9/master/ICD9.txt), simply take the three first characters
        #if not map_ICD9_to_CCS:
          #  ICD_code = ICD_code[:4]  # No CCS mapping, get the first alphanumeric four letters only


        if hadm_id in mapping:
            mapping[hadm_id].append(ICD_code.strip())
        else:
            mapping[hadm_id]= [ICD_code.strip()]  
    mimicFile.close()
    print ('-Number of null ICD9 codes in file ' + fileName + ': ' + str(number_of_null_ICD9_codes))
    print ('-Number of null ICD10 codes in file ' + fileName + ': ' + str(number_of_null_ICD10_codes))
    #print ('-Number of diagnosis codes in file ' + fileName + ': ' + str(len(codes)))
    return mapping

In [84]:
infd = gzip.open(prescriptionFile, 'r')
infd.readline()
for line in infd:
    tokens = line.decode('utf-8').strip().split(',')
    print(tokens)
    break

['10000032', '22595853', '11700683', '10000032-34', '34', 'P76JEQ', '2180-05-07 01:00:00', '2180-05-07 22:00:00', 'MAIN', 'Acetaminophen', 'APAP500', '004490', '00904198861', '500mg Tablet', '', '500', 'mg', '1', 'TAB', '', 'PO/NG']


In [81]:
adm.head()

,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10000032,22595853,2180-05-06 22:23:00,2180-05-07 17:15:00,NaN,URGENT,P874LG,TRANSFER FROM HOSPITAL,HOME,Other,ENGLISH,WIDOWED,WHITE,2180-05-06 19:17:00,2180-05-06 23:30:00,0
1,10000032,22841357,2180-06-26 18:27:00,2180-06-27 18:49:00,NaN,EW EMER.,P09Q6Y,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-06-26 15:54:00,2180-06-26 21:31:00,0
2,10000032,25742920,2180-08-05 23:44:00,2180-08-07 17:50:00,NaN,EW EMER.,P60CC5,EMERGENCY ROOM,HOSPICE,Medicaid,ENGLISH,WIDOWED,WHITE,2180-08-05 20:58:00,2180-08-06 01:44:00,0
3,10000032,29079034,2180-07-23 12:35:00,2180-07-25 17:55:00,NaN,EW EMER.,P30KEH,EMERGENCY ROOM,HOME,Medicaid,ENGLISH,WIDOWED,WHITE,2180-07-23 05:54:00,2180-07-23 14:00:00,0
4,10000068,25022803,2160-03-03 23:16:00,2160-03-04 06:26:00,NaN,EU OBSERVATION,P51VDL,EMERGENCY ROOM,NaN,Other,ENGLISH,SINGLE,WHITE,2160-03-03 21:55:00,2160-03-04 06:26:00,0


In [99]:
pres = pd.read_csv(prescriptionFile,low_memory=False)
pres.head()

,subject_id,hadm_id,pharmacy_id,poe_id,poe_seq,order_provider_id,starttime,stoptime,drug_type,drug,...,gsn,ndc,prod_strength,form_rx,dose_val_rx,dose_unit_rx,form_val_disp,form_unit_disp,doses_per_24_hrs,route
0,10000032,22595853,11700683,10000032-34,34.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Acetaminophen,...,004490,9.041989e+08,500mg Tablet,NaN,500,mg,1,TAB,NaN,PO/NG
1,10000032,22595853,14779570,10000032-22,22.0,P76JEQ,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Sodium Chloride 0.9% Flush,...,NaN,0.000000e+00,10 mL Syringe,NaN,3,mL,0.3,SYR,3.0,IV
2,10000032,22595853,19796602,10000032-50,50.0,P260SK,2180-05-08 08:00:00,2180-05-07 22:00:00,MAIN,Furosemide,...,008209,5.107901e+10,40mg Tablet,NaN,40,mg,1,TAB,1.0,PO/NG
3,10000032,22595853,20256254,10000032-32,32.0,P76JEQ,2180-05-07 01:00:00,2180-05-07 22:00:00,MAIN,Raltegravir,...,063231,6.022761e+06,400 mg Tablet,NaN,400,mg,1,TAB,2.0,PO
4,10000032,22595853,28781051,10000032-27,27.0,P76JEQ,2180-05-07 00:00:00,2180-05-07 22:00:00,MAIN,Heparin,...,006549,6.332303e+10,5000 Units / mL- 1mL Vial,NaN,5000,UNIT,1,mL,3.0,SC


In [100]:
pres.columns.get_loc('drug')

9

In [101]:
mimicFile = gzip.open(prescriptionFile, 'r')  # subject_id,hadm_id,gsn,ndc,drug
mimicFile.readline()
number_of_null_NDC_codes = 0
for line in mimicFile:
        #print(line)#   0  ,     1    ,    2   ,   3  ,    4
        #break
        tokens = line.decode('utf-8').strip().split(',')
        #print(tokens)
        print(tokens[9])
        break

Acetaminophen


In [111]:
def get_drugs_from_mimic_file(fileName, choice ='ndc'):
    """
    This creates a hospital to list of drugs mapping and a drug to description ( name ) map
    
    inputs:
    fileNAME : path to the procedure file
    choice : drug codification to choose from
    
    outputs:
    drugDescription : dict that maps drug codes to their name
    mapping : dict that maps hospital admissions to a list of ndc/gsn drug codes
    
    
    """
    mapping = {}
    drugDescription = {}
    mimicFile = gzip.open(fileName, 'r')  # subject_id,hadm_id,gsn,ndc,drug
    mimicFile.readline()
    number_of_null_NDC_codes = 0
    try:
        for line in mimicFile:
            #print(line)#   0  ,     1    ,    2   ,   3  ,    4
            #break
            tokens = line.decode('utf-8').strip().split(',')
            #print(tokens)
            hadm_id = int(tokens[1])
            if choice =='ndc':                        #code : Total Number of NDC code 5912
                drug_code = tokens[12]   
            else:    
                drug_code = tokens[11]                    #code : Total Number of gsn code 3081

            drug_code = drug_code.strip()  

            drug_code = 'DR'+'_'+drug_code
            if hadm_id in mapping:
                mapping[hadm_id].append(drug_code.strip())
            else:
                #mapping[hadm_id]=set()           #use set to avoid repetitions
                #mapping[hadm_id].add(drug_code.strip())
                mapping[hadm_id]=[drug_code.strip()]
                
            if drug_code not in drugDescription:
                drugDescription[drug_code] = tokens[9]
                
    except Exception as e:
        print(line)
        print(e)
    #for hadm_id in mapping.keys():
        #mapping[hadm_id] = list(mapping[hadm_id])   #convert to list, as the rest of the codes expects
    mimicFile.close()
    return drugDescription, mapping

In [114]:
def load_mimic_data(choice ='ndc'):
    """
    inputs:
    choice : the type of drug description to choose 
    output:
    subject_idAdmMap : dict that maps subject ids to hostpital admissions
    admDxMap : dict that maps hospital admission ids to a list of ICD-9/ICD-10 diagnosis codes
    admPxMap : dict that maps hospital admission ids to a list of ICD-9/ICD-10 procedure codes
    admDrugMap : dict that maps hospital admissions to a list of ndc/gsn drug codes
    drugDescription : dict that maps drug codes to their name
    """
    print ('Building subject_id-admission mapping, admission-date mapping')
    previous_subject = 0
    previous_admission = 0
    subject_idAdmMap = {}
    admDateMap = {}
    subject_idStatic = {}   # adm type, Insurance , ethnicity , marital status
    infd = gzip.open(admissionFile, 'r')
    infd.readline()
    for line in infd:
        tokens = line.decode('utf-8').strip().split(',')
        subject_id = int(tokens[0])
        hadm_id = int(tokens[1])
        #admTime = datetime.strptime(tokens[2], '%Y-%m-%d %H:%M:%S')
        #admTime = tokens[3]
        #admDateMap[hadm_id] = admTime
        #subject_idStatic[subject_id] = [convert_binary_to_Int(tokens)]
        if subject_id in subject_idAdmMap: 
            subject_idAdmMap[subject_id].add(hadm_id)
        else: 
            subject_idAdmMap[subject_id] = set()
            subject_idAdmMap[subject_id].add(hadm_id)
    for subject_id in subject_idAdmMap.keys():
        subject_idAdmMap[subject_id] = list(subject_idAdmMap[subject_id])  
    infd.close()

    print ('Building admission-diagnosis mapping')
    admDxMap = get_ICDs_from_mimic_file(diagnosisFile)

    print ('Building admission-procedure mapping')
    admPxMap = get_ICDs_from_mimic_file(procedureFile, isdiagnosis=False)

    print ('Building admission-drug mapping')
    drugDescription, admDrugMap = get_drugs_from_mimic_file(prescriptionFile, choice)
    return subject_idAdmMap,admDxMap,admPxMap,admDrugMap,drugDescription


In [ ]:
def updateAdmCodeList(subject_idAdmMap,admDxMap,admPxMap,admDrugMap):
    """
    This function recreates the admissions, procedure, diagnosis 
    """
    adDx = {}
    adPx = {}
    adDrug={}
    for subject_id, admIdList in subject_idAdmMap.items():
        for admId in admIdList:
            adDx[admId] = admDxMap[admId]
            adPx[admId] =admPxMap[admId]
            #adAge[admId] = admAgeMap[admId]
            adDrug[admId] =admDrugMap[admId]
            
    return adDx,adPx,adDrug

In [ ]:
def clean_data(subject_idAdmMap,admDxMap,admPxMap,admDrugMap, min_admissions_threshold = 2):
    # removing the subject_id which are not present in diagnostic code but present in procedure and vice versa
    print("Cleaning data...")
    subDelList = []

    print("Removing patient records who does not have all three medical codes for an admission")
    for subject_id,hadm_ids in  subject_idAdmMap.items():
        for hadm_id in hadm_ids:
            if (hadm_id not in admDxMap.keys()):
                subDelList.append(subject_id)
            if (hadm_id not in admPxMap.keys()):
                subDelList.append(subject_id)
            if (hadm_id not in admDrugMap.keys()):
                subDelList.append(subject_id)

    subDelList = list(set(subDelList))       
    print(f"Number of subject_ids to be deleted :{len(subDelList)} ")

    for subject_id_to_rm in subDelList:
        del subject_idAdmMap[subject_id_to_rm]

    #print(f"Number of subject_ids aftr cleaning :{len(subject_idAdmMap)} ")  
            
    adDx,adPx,adDrug=updateAdmCodeList(subject_idAdmMap,admDxMap,admPxMap,admDrugMap)

    display(subject_idAdmMap,adDx,adPx,adDrug)
    # removing patient who made less than 2 admissions

    print(f"Removing patient who made less than {min_admissions_threshold} admissions")
    pidMap = {}
    adm = []
    subDelList=[]
    subject_idAdmMap1 = subject_idAdmMap
    for pid, admIdList in subject_idAdmMap.items():
        if len(admIdList) < min_admissions_threshold:
            subDelList.append(pid)
            continue

    for i in subDelList:
        del subject_idAdmMap[i]  

    adDx,adPx,adDrug=updateAdmCodeList(subject_idAdmMap,adDx,adPx,adDrug)   
    display(subject_idAdmMap,adDx,adPx,adDrug)  
    return subject_idAdmMap,adDx,adPx,adDrug

# Done porting: 

In [ ]:
mimic_iv_path = 'mimic-iv-2.2/hosp'
CCSRDX_file = args.CCSRDX_file
CCSRPCS_file = args.CCSRPCS_file
D_CCSR_Ref_file = args.D_CCSR_Ref_file
P_CCSR_Ref_file = args.P_CCSR_Ref_file
CCSDX_file = args.CCSDX_file
CCSPX_file = args.CCSPX_file
min_dx = args.min_dx
min_px = args.min_px
min_drg = args.min_drg
seqLength = args.seqLength
threshold = args.threshold
print("Loading the data...")
subject_idAdmMap,admDxMap,admPxMap,admDrugMap,drugDescription = load_mimic_data()
print("\n Completed...")

# Working on

In [ ]:
#stage 2 and 3
print("\n Cleaning data...")
subject_idAdmMap,adDx,adPx,adDrug = clean_data(subject_idAdmMap,admDxMap,admPxMap,admDrugMap)

# To dos 

In [8]:


print("\n Completed...")
#stage 4
print("\n Mapping ICD data to CCS and CCSR...")
adDx,adPx,codeDescription = icd_mapping(CCSRDX_file,CCSRPCS_file,CCSDX_file,CCSPX_file,D_CCSR_Ref_file,P_CCSR_Ref_file,adDx,adPx,adDrug,drugDescription)
print("\n Completed...")
#stage 5
print("\n Trimming the codes assigned per visit based on a threshold...")
adDx,adPx,adDrug= trim(min_dx,min_px,min_drg,adDx,adPx,adDrug)
print("\n Completed...")

print("\n Building the data..")
newSeqs,types=buildData(subject_idAdmMap,adDx,adPx,adDrug)
#stage 6
print(f"\n removing the code whose occurence is less than a certain {threshold}")
updatedSeqs,types,reverseTypes = removeCode(newSeqs,types,threshold=threshold)
# outFile - is a folder path in the working directory where the data is going to get stored
outFile = os.path.join('outputData','originalData')
print("\n Save the data before formmating based on the task")
saveFiles(updatedSeqs,types,codeDescription)
codeType = generateCodeTypes(outFile,reverseTypes)
seqs,types,codeType,reverseTypes = load_data(outFile)
print("\n Completed...")
print("\n Preparing data for Trajectory Forecasting....")
# sequence length threshold  -mn
newPairs = formatData(seqs,dataFormat = 'TF',mn = seqLength)
diagnosisOutputFile = os.path.join('outputData','TF','Inp_d_p_dr_out_d')
diagnosisProcedureOutputFile = os.path.join('outputData','TF','Inp_d_p_dr_out_d_p')
AllOutputFile = os.path.join('outputData','TF','Inp_d_p_dr_out_d_p_dr')

print(f"\n Remove certain codes from output for different data formats")
AllUpdPair,AllOutTypes= resetIntegerOutput(updateOutput(newPairs.copy(),codeType,diagnosis=0,procedure=0,drugs =0,all =1))
diagnosisUpdPair,diagnosisOutTypes= resetIntegerOutput(updateOutput(newPairs.copy(),codeType,diagnosis=0,procedure=1,drugs =1,all =0))
diagnosisProcedureUpdPair,diagnosisProcedureOutTypes= resetIntegerOutput(updateOutput(newPairs.copy(),codeType,diagnosis=0,procedure=0,drugs =1,all =0))

print(f"\n total # S1 records : {len(diagnosisUpdPair)}\n total # S2 records :{len(diagnosisProcedureUpdPair)}\n total # S3 records :{len(AllUpdPair)}")
print(f"\n total Dx codes:{len(diagnosisOutTypes)} \n  total Dx,Px codes:{len(diagnosisProcedureOutTypes)} \n total Dx,Px,Rx codes:{len(AllOutTypes)}")
print("\n Storing all the information related to Trajectory Forecasting...")


storeFiles(diagnosisUpdPair,diagnosisOutTypes,codeType,types,reverseTypes,diagnosisOutputFile)
storeFiles(diagnosisProcedureUpdPair,diagnosisProcedureOutTypes,codeType,types,reverseTypes,diagnosisProcedureOutputFile)
storeFiles(AllUpdPair,AllOutTypes,codeType,types,reverseTypes,AllOutputFile)
print("\n Completed...")

print("\nPreparing data for Sequential disease prediction....")
newPairs = formatData(seqs,dataFormat = 'SDP',mn =500)
diagnosisOutputFile = os.path.join('outputData','SDP','Inp_d_p_dr_out_d')

print(f"\n\n Remove certain codes from output for different data formats")
diagnosisUpdPair,diagnosisOutTypes= resetIntegerOutput(updateOutput(newPairs.copy(),codeType,diagnosis=0,procedure=1,drugs =1,all =0))

print(f"\n total # records: {len(diagnosisUpdPair)} \n total # of codes: {len(diagnosisOutTypes)}")

print("\n Storing all the information related to TSequential disease prediction...")
storeFiles(diagnosisUpdPair,diagnosisOutTypes,codeType,types,reverseTypes,diagnosisOutputFile)
print("\n Completed...")
print("\n All the preprocessing step has been completed, Now use the data in the outputData folder to build the model...")


parser.add_argument('--mimic_iv_path',default='data', type=str,help="Path of mimic IV CSV files where the queried data is stored")

parser.add_argument('--CCSRDX_file',default='DXCCSR_v2021-2/DXCCSR_v2021-2.csv', type=str,help="Path of diagnosis based CCSR files")
parser.add_argument('--CCSRPCS_file',default='PRCCSR_v2021-1/PRCCSR_v2021-1.csv', type=str,help="Path of procedure based CCSR files")

parser.add_argument('--D_CCSR_Ref_file',default='DXCCSR_v2021-2/DXCCSR-Reference-File-v2021-2.xlsx', type=str,help="Path of diagnosis based CCSR Reference file ")
parser.add_argument('--P_CCSR_Ref_file',default='PRCCSR_v2021-1/PRCCSR-Reference-File-v2021-1.xlsx', type=str,help="Path of procedure based CCSR Reference file")


parser.add_argument('--CCSDX_file',default='Single_Level_CCS_2015/$dxref 2015.csv', type=str,help="Path of diagnosis based CCS files")
parser.add_argument('--CCSPX_file',default='Single_Level_CCS_2015/$prref 2015.csv', type=str,help="Path of procedure based CCS files")

parser.add_argument('--min_dx',default=80, type=int,help="Minimum diagnosis code assigned per visit")
parser.add_argument('--min_px',default=80, type=int,help="Minimum procedure code assigned per visit")
parser.add_argument('--min_drg',default=80, type=int,help="Minimum drug/medication code assigned per visit")

parser.add_argument('--threshold',default=5, type=int,help="Remove the code whose frequency  is less than the threshold")
parser.add_argument('--seqLength',default=500, type=int,help="maximum sequence length of each sequence in input and output")

NameError: name 'args' is not defined